## Generate response

In [1]:
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.llms import HuggingFaceHub
import os
from langchain.embeddings import VoyageEmbeddings
from dotenv import load_dotenv

In [2]:
CHROMA_ADVICE_PATH = "chroma\\advice"
CHROMA_TRIBUNAL_PATH = "chroma\\tribunal"
repo_id = "google/flan-t5-xxl"
load_dotenv()

True

In [3]:
SUMMARY_PROMPT_TEMPLATE = """

Summarise: {question}
"""

In [4]:
QUESTION_PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question and provide a rationale based on the above context: {question}
"""

In [5]:
# Submit propmt to model
model = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 128}
)

c:\Users\Chris\anaconda3\envs\RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Chris\anaconda3\envs\RAG\Lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
original_query_text = "For what reasons can a landlord terminate a tenancy?"

In [7]:
# Create query prompt
prompt_template = ChatPromptTemplate.from_template(SUMMARY_PROMPT_TEMPLATE)
prompt = prompt_template.format(question = original_query_text)
query_text = model.predict(prompt)

In [8]:
# Prepare the DB.
embedding_function = VoyageEmbeddings()
advice_db = Chroma(persist_directory=CHROMA_ADVICE_PATH, embedding_function=embedding_function)

In [13]:
advice_results = advice_db.similarity_search_with_relevance_scores(query_text, k=3)

In [15]:
advice_results[0]

(Document(page_content='A landlord can apply to the Tenancy Tribunal to end a tenancy for any of the reasons outlined below. The tenant can oppose this application. In most cases the tenant will need to show that the grounds are not valid.\n\nIf the landlord is successful then the Tribunal will decide what the notice period is.', metadata={'link': 'https://tenant.aratohu.nz/ways-of-ending-a-tenancy/ending-a-tenancy-via-the-tenancy-tribunal/', 'source': 'data\\aratohu\\tribunal ending tenancy.md', 'start_index': 0}),
 0.7579436104782342)

In [16]:
advice_db.get(where={"source" : "data\\aratohu\\abandoned goods.md"})

{'ids': ['effc9653-ac1b-11ee-9ba6-a86bad711010',
  'effc9654-ac1b-11ee-992e-a86bad711010',
  'effc9655-ac1b-11ee-adb5-a86bad711010'],
 'embeddings': None,
 'metadatas': [{'link': 'https://tenant.aratohu.nz/after-the-tenancy/abandoned-goods/',
   'source': 'data\\aratohu\\abandoned goods.md',
   'start_index': 0},
  {'link': 'https://tenant.aratohu.nz/after-the-tenancy/abandoned-goods/',
   'source': 'data\\aratohu\\abandoned goods.md',
   'start_index': 301},
  {'link': 'https://tenant.aratohu.nz/after-the-tenancy/abandoned-goods/',
   'source': 'data\\aratohu\\abandoned goods.md',
   'start_index': 533}],
 'documents': ['It’s the tenant’s responsibility to remove their property at the end of the tenancy. If they don’t do this, their belongings may end up being disposed of or sold.\nIf the tenant leaves goods behind after the end of the tenancy, there are rules the landlord must follow in dealing with them (s 62 RTA).\nThe landlord can dispose of perishable items (eg, food) immediately

In [12]:
advice_db.get(where={
    "$and": [
        {
            "start_index" : { 
                "$gt" : 301
            }
        },
        {
            "source": {
                "$eq": "data\\aratohu\\abandoned goods.md"
            }
        }
         ]})

{'ids': ['effc9655-ac1b-11ee-adb5-a86bad711010'],
 'embeddings': None,
 'metadatas': [{'link': 'https://tenant.aratohu.nz/after-the-tenancy/abandoned-goods/',
   'source': 'data\\aratohu\\abandoned goods.md',
   'start_index': 533}],
 'documents': ['If the landlord doesn’t manage to contact the tenant, or the tenant doesn’t pick up their property as agreed, then the landlord can either: apply to the Tenancy Tribunal for an order determining what to do with the belongings (ie, order the goods to be disposed of, returned to the tenant, or sold by the landlord), or store, sell or dispose of the tenant’s belongings in accordance with the required process. See Tenancy Services information about abandoned goods for more information.'],
 'uris': None,
 'data': None}

In [15]:
# Prepare the prompt
advice_results = advice_db.similarity_search_with_relevance_scores(query_text, k=3)


results = advice_results 

print(results)

if len(results) == 0 or results[0][1] < 0.4:
    print(f"Unable to find matching results.")
else:
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(QUESTION_PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(prompt)

[]
Unable to find matching results.


In [55]:
response_text = model.predict(prompt)

In [56]:
sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"
print(formatted_response)

Response: Sometimes the landlord will be happy to mow the lawns and weed the garden, or they will hire a gardener to do it. If this is the case, it’s best if this arrangement is put in writing, preferably in the tenancy agreement at the start of the tenancy. This way, there is a clear record of the
Sources: ['data\\aratohu\\outgoings and paying bills.md', 'data\\aratohu\\unenforceable and unlawful tenancy conditions.md', 'data\\aratohu\\types of maintenance.md', 'data\\tribunal\\tribunal_order_12.md', 'data\\tribunal\\tribunal_order_27.md', 'data\\tribunal\\tribunal_order_33.md']
